In [ ]:
import pandas as pd
import os
import pandas as pd
from difflib import SequenceMatcher

In [ ]:
# Define the input and output file paths
input_file_path = 'passes.csv'  # Replace with your actual file path
output_file_path = 'filtered_passes_binary.csv'

In [ ]:
# Ensure the file exists
if not os.path.exists(input_file_path):
    print(f"Error: The file '{input_file_path}' does not exist.")
else:
    try:
        # Read the original CSV file
        df = pd.read_csv(input_file_path)

        # Check if the necessary columns exist
        main_columns = ['time_start', 'time_end', 'sender_id', 'receiver_id']
        target_columns = [f'x_{i}' for i in range(1, 29)]  # x1 to x28
        all_columns = main_columns + target_columns

        missing_columns = [col for col in all_columns if col not in df.columns]
        if missing_columns:
            print(f"Error: The following columns are missing: {', '.join(missing_columns)}")
        else:
            # Function to create the binary sequence for empty and non-empty cells
            def create_binary_sequence(row):
                return ''.join(['0' if pd.isna(row[col]) else '1' for col in target_columns])

            # Apply the function to create the binary sequence column
            df['empty_columns_sequence'] = df[target_columns].apply(create_binary_sequence, axis=1)

            # Select the required columns for the output
            filtered_df = df[main_columns + ['empty_columns_sequence']]

            # Save the filtered DataFrame to a new CSV file
            filtered_df.to_csv(output_file_path, index=False)
            print(f"Filtered file saved successfully to {output_file_path}")

    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
# making a direcotry
os.mkdir('this')

In [ ]:
import pandas as pd
from difflib import SequenceMatcher

def similar(a, b):
    """Check similarity of two strings using SequenceMatcher."""
    return SequenceMatcher(None, a, b).ratio()

def process_csv(input_file, threshold=0.6):
    # Load the data
    df = pd.read_csv(input_file)
    match_count = 0

    while not df.empty:
        match_count += 1
        first_row = df.iloc[0]
        current_time = first_row["time_end"]
        current_receiver = first_row["receiver_id"]
        current_sequence = first_row["empty_columns_sequence"]

        # Start creating the match group
        match_rows = [first_row.name]  # Store index of matched rows

        while True:
            potential_matches = df[df["time_start"] == current_time]
            if not potential_matches.empty:
                matched_row = None
                for _, row in potential_matches.iterrows():
                    if row["sender_id"] == current_receiver:
                        matched_row = row
                        break
                if matched_row is None:  # Check empty_columns_sequence if no direct match
                    for _, row in potential_matches.iterrows():
                        if similar(row["empty_columns_sequence"], current_sequence) >= threshold:
                            matched_row = row
                            break

                if matched_row is not None:
                    match_rows.append(matched_row.name)
                    current_time = matched_row["time_end"]
                    current_receiver = matched_row["receiver_id"]
                    current_sequence = matched_row["empty_columns_sequence"]
                else:
                    break  # No further matches found
            else:
                break  # No potential matches

        # Save the match to a new CSV file
        match_df = df.loc[match_rows]
        match_df.to_csv(f"this/match_{match_count}.csv", index=False)

        # Drop the matched rows from the main DataFrame
        df = df.drop(index=match_rows).reset_index(drop=True)

# Run the script
process_csv("filtered_passes_binary.csv")
